# Mobile Net

In [ ]:
# Install necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Helper function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for img_path in image_paths:
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, target_size)  # Resize to match MobileNet input
            image = preprocess_input(image)  # Preprocess for MobileNet
            images.append(image)
    return np.array(images)

# Define paths and reference card details
test_images_path = '/content/drive/MyDrive/dataset/test_images'  # Update with test images path
dataset_path = '/content/drive/MyDrive/course data( all files needed for students )/Anemia_dataset_train.xlsx'  # dataset path

# Load Hb_Value and prepare dataset
print("Loading Hb_Value data...")
right_eye_data = pd.read_excel(dataset_path, sheet_name='Right_Eye_Data')

# Match images with Hb values using filenames
print("Matching image filenames with Hb values...")
image_filenames = os.listdir(test_images_path)
hb_filenames = right_eye_data["Right_eye_Images"]  # Adjust this to the actual column containing image filenames

# Filter valid entries
valid_image_paths = []
valid_hb_values = []

for img in image_filenames:
    if img in hb_filenames.values:
        valid_image_paths.append(os.path.join(test_images_path, img))
        valid_hb_values.append(right_eye_data.loc[right_eye_data["Right_eye_Images"] == img, "Hb_Value"].values[0])

# Check for missing files
missing_files = [filename for filename in hb_filenames if filename not in image_filenames]
if missing_files:
    print(f"Warning: Missing files detected! {missing_files}")

# Preprocess valid images
print("Preprocessing valid images...")
images = load_and_preprocess_images(valid_image_paths)
hb_values = pd.Series(valid_hb_values)  # Convert to a Pandas Series for compatibility

# Split data into training and testing sets
print("Splitting data into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(images, hb_values, test_size=0.2, random_state=42)

# Load MobileNet model for feature extraction
print("Loading MobileNet model...")
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Pooling layer
    Dropout(0.3),  # Prevent overfitting
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.3),
    Dense(1, activation='linear')  # Single output for Hb value prediction
])

# Compile the model
print("Compiling the model...")
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])

# Train the model
print("Training the model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=16,
    verbose=1
)

# Evaluate the model
print("Evaluating the model...")
y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = np.mean(np.abs(y_test - y_pred))  # Calculate Mean Absolute Error
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R² Score: {r2}")


# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot Actual vs Predicted Hb Values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7, label="Predicted vs Actual")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color="red", linestyle="--", label="Ideal Fit")
plt.title("Actual vs Predicted Hb Values")
plt.xlabel("Actual Hb Value")
plt.ylabel("Predicted Hb Value")
plt.legend()
plt.grid(True)
plt.show()

# Residual Plot
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.7, label="Residuals")
plt.axhline(0, color="red", linestyle="--", label="Zero Error Line")
plt.title("Residual Plot")
plt.xlabel("Predicted Hb Value")
plt.ylabel("Residual (Actual - Predicted)")
plt.legend()
plt.grid(True)
plt.show()

# Save predictions to a CSV file
output_path = '/content/mobilenet_predictions.csv'
results_df = pd.DataFrame({
    "Actual_Hb_Value": y_test,
    "Predicted_Hb_Value": y_pred
})
results_df.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")

# Efficient Net

In [ ]:
# Install necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Helper function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(224, 224)):
    images = []
    for img_path in image_paths:
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, target_size)  # Resize to match EfficientNet input
            image = preprocess_input(image)  # Preprocess for EfficientNet
            images.append(image)
    return np.array(images)

# Define paths and reference card details
test_images_path = '/content/drive/MyDrive/dataset/test_images'  # Update with test images path
dataset_path = '/content/drive/MyDrive/course data( all files needed for students )/Anemia_dataset_train.xlsx'  # dataset path

# Load Hb_Value and prepare dataset
print("Loading Hb_Value data...")
right_eye_data = pd.read_excel(dataset_path, sheet_name='Right_Eye_Data')

# Match images with Hb values using filenames
print("Matching image filenames with Hb values...")
image_filenames = os.listdir(test_images_path)
hb_filenames = right_eye_data["Right_eye_Images"]  # Adjust this to the actual column containing image filenames

# Filter valid entries
valid_image_paths = []
valid_hb_values = []

for img in image_filenames:
    if img in hb_filenames.values:
        valid_image_paths.append(os.path.join(test_images_path, img))
        valid_hb_values.append(right_eye_data.loc[right_eye_data["Right_eye_Images"] == img, "Hb_Value"].values[0])

# Check for missing files
missing_files = [filename for filename in hb_filenames if filename not in image_filenames]
if missing_files:
    print(f"Warning: Missing files detected! {missing_files}")

# Preprocess valid images
print("Preprocessing valid images...")
images = load_and_preprocess_images(valid_image_paths)
hb_values = pd.Series(valid_hb_values)  # Convert to a Pandas Series for compatibility

# Split data into training and testing sets
print("Splitting data into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(images, hb_values, test_size=0.2, random_state=42)

# Load EfficientNet model for feature extraction
print("Loading EfficientNet model...")
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Pooling layer
    Dropout(0.3),  # Prevent overfitting
    Dense(128, activation='relu'),  # Fully connected layer
    Dropout(0.3),
    Dense(1, activation='linear')  # Single output for Hb value prediction
])

# Compile the model
print("Compiling the model...")
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])

# Train the model
print("Training the model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs= 100,
    batch_size=16,
    verbose=1
)

# Evaluate the model
print("Evaluating the model...")
y_pred = model.predict(X_test).flatten()
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
mae = np.mean(np.abs(y_test - y_pred))  # Calculate Mean Absolute Error
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R² Score: {r2}")

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot Actual vs Predicted Hb Values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7, label="Predicted vs Actual")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color="red", linestyle="--", label="Ideal Fit")
plt.title("Actual vs Predicted Hb Values")
plt.xlabel("Actual Hb Value")
plt.ylabel("Predicted Hb Value")
plt.legend()
plt.grid(True)
plt.show()

# Residual Plot
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.7, label="Residuals")
plt.axhline(0, color="red", linestyle="--", label="Zero Error Line")
plt.title("Residual Plot")
plt.xlabel("Predicted Hb Value")
plt.ylabel("Residual (Actual - Predicted)")
plt.legend()
plt.grid(True)
plt.show()

# Save predictions to a CSV file
output_path = '/content/efficientnet_predictions.csv'
results_df = pd.DataFrame({
    "Actual_Hb_Value": y_test,
    "Predicted_Hb_Value": y_pred
})
results_df.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")
